## Import Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

## Import

In [ ]:
%%capture
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose, ZeroPadding2D
from tensorflow.keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization,Reshape
from tensorflow.keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU, Multiply, add,multiply
from tensorflow.keras.utils import plot_model
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot
import pickle
from time import time
import shutil
import numpy as np
import os
import tensorflow as tf
from tensorflow.python.framework import graph_util
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.optimizers import Adam, RMSprop,SGD
import glob
from numpy import expand_dims
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import random
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam, RMSprop,SGD

## MDenseNet - keras version

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

from keras.applications import imagenet_utils
from keras.applications.imagenet_utils import decode_predictions


def dense_block(x, blocks, name):

    for i in range(blocks):
        x = conv_block(x, 32, name=name + '_block' + str(i + 1))
    return x


def transition_block(x, reduction, name):

    bn_axis = 3
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_bn')(x)
    x = Activation('relu', name=name + '_relu')(x)
    x = Conv2D(int(K.int_shape(x)[bn_axis] * reduction), 1,
                      use_bias=False,
                      name=name + '_conv')(x)
    x = AvgPool2D(2, strides=2, name=name + '_pool')(x)
    return x

def add_block(_x1,_x2):
    # print('_x1',K.int_shape(_x1))
    # print('_x2',K.int_shape(_x2))
    size = int(K.int_shape(_x1)[1] / K.int_shape(_x2)[1])
    _x2 = UpSampling2D(size=(size,size))(_x2)
    sc = Conv2D(K.int_shape(_x2)[-1], 1, strides=1,padding='same')(_x1)
    sc = BatchNormalization()(sc)
    output = Add()([_x2,sc])
    output = ReLU()(output)
    output = MaxPool2D(size,padding='same')(output)
    # print('output',K.int_shape(output))
    return output

def conv_block(x, growth_rate, name):
    bn_axis = 3
    x1 = BatchNormalization(axis=bn_axis,
                                   epsilon=1.001e-5,
                                   name=name + '_0_bn')(x)
    x1 = ReLU()(x1)
    x1 = Conv2D(4 * growth_rate, 1,
                       use_bias=False,
                       name=name + '_1_conv')(x1)
    x1 = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                   name=name + '_1_bn')(x1)
    x1 = ReLU()(x1)
    x1 = SeparableConv2D(growth_rate, 3,
                       padding='same',
                       use_bias=False,
                       name=name + '_2_conv')(x1)
    x = Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])
    return x


def DenseNet(blocks,
             include_top=True,
             weights='imagenet',
             input_tensor=None,
             input_shape=None,
             pooling=None,
             classes=1000,
             **kwargs):

    if not (weights in {'imagenet', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')


    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    bn_axis = 3 

    x = ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
    x = Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(x)
    x = BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name='conv1/bn')(x)
    x = ReLU()(x)
    x = ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x0 = MaxPool2D(3, strides=2, name='pool1')(x)

    # if abs == True:
    x1 = dense_block(x0, blocks[0], name='conv2')
    x1 = transition_block(x1, 0.5, name='pool2')
    x1 = add_block(x0,x1)

    x2 = dense_block(x1, blocks[1], name='conv3')
    x2 = transition_block(x2, 0.5, name='pool3')
    x2 = add_block(x0,x2)
    x2 = add_block(x1,x2)

    x3 = dense_block(x2, blocks[2], name='conv4')
    x3 = transition_block(x3, 0.5, name='pool4')
    x3 = add_block(x0,x3)
    x3 = add_block(x1,x3)
    x = add_block(x2,x3)

    x = dense_block(x, blocks[3], name='conv5')


    x = BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name='bn')(x)
    x = Activation('relu', name='relu')(x)

    x = GlobalAvgPool2D(name='avg_pool')(x)
    x = Dense(classes, activation='softmax', name='fc'+str(classes))(x)


    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = keras_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input

    # Create model.
    if blocks == [6, 12, 24, 16]:
        model = Model(inputs, x, name='densenet121abs')
    elif blocks == [6, 12, 32, 32]:
        model = Model(inputs, x, name='densenet169abs')
    elif blocks == [6, 12, 48, 32]:
        model = Model(inputs, x, name='densenet201abs')
    else:
        model = Model(inputs, x, name='densenetabs')
    return model


def DenseNet121abs(include_top=True,
                weights='imagenet',
                input_tensor=None,
                input_shape=None,
                pooling=None,
                classes=1000,
                
                **kwargs):
    return DenseNet([6, 12, 24, 16],
                    include_top, weights,
                    input_tensor, input_shape,
                    pooling, classes,
                    **kwargs)


def DenseNet169abs(include_top=True,
                weights='imagenet',
                input_tensor=None,
                input_shape=None,
                pooling=None,
                classes=1000,
                
                **kwargs):
    return DenseNet([6, 12, 32, 32],
                    include_top, weights,
                    input_tensor, input_shape,
                    pooling, classes, 
                    **kwargs)


def DenseNet201abs(include_top=True,
                weights='imagenet',
                input_tensor=None,
                input_shape=None,
                pooling=None,
                classes=1000,
                
                **kwargs):
    return DenseNet([6, 12, 48, 32],
                    include_top, weights,
                    input_tensor, input_shape,
                    pooling, classes,
                    **kwargs)

## DenseNet - origin keras version

In [ ]:
def densenet_keras(n_conv,weights = None ,input_shape = (224,224,3)):
  '''n_conv: number of conv layer: 121, 169 or 201
      weights: None or 'Imagenet'
      input_shape: tuple example (224,224,3)'''
  if n_conv == 121:
    modeltest = tensorflow.keras.applications.DenseNet121(include_top=False, weights=weights, input_shape=input_shape)
  elif n_conv == 169:
    modeltest = tensorflow.keras.applications.DenseNet169(include_top=False, weights=weights, input_shape=input_shape)
  elif n_conv == 201:
    modeltest = tensorflow.keras.applications.DenseNet201(include_top=False, weights=weights, input_shape=input_shape)
  # # modeltest.summary()
  x = GlobalAvgPool2D()(modeltest.output)
  output = Dense(n_classes,activation='softmax')(x)
  model = Model(modeltest.input, output,name = modeltest.name+'_norm')
  return model

In [ ]:
input_shape = 224,224,3

n_classes = 100 # 7 if using bamboo dataset
K.clear_session()
global graph

# Model Densenet121abs, Densenet169abs, Densenet201abs
model = DenseNet121abs(include_top=False,weights=None,input_shape=(224,224,3),classes=n_classes)
print('Model name: {}\nModel params: {} '.format(model.name, model.count_params()))

# Calculate FLOPs
input = np.random.randn(1, *input_shape)
graph = tf.compat.v1.get_default_graph()
sess = tf.compat.v1.Session(graph=graph)
model._make_predict_function()
sess.run(tf.compat.v1.global_variables_initializer())
flops = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation())
print(model.name,' FLOP = ', flops.total_float_ops)

sgd = SGD(lr = 0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
adam = Adam(lr=0.001)
rmsp = RMSprop(lr=0.1)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

## Dataset

In [ ]:
from zipfile import ZipFile

def extract(file_name_data,fp='data'):
  with ZipFile(file_name_data,'r') as zip:
    zip.extractall(fp)

# For bamboo dataset
# extract('/content/drive/MyDrive/dataset/DATA.zip')

# For mini-imagenet
extract('/content/drive/My Drive/Slice-ImageNet/Tiny-ImageNet2.zip','mini-imagenet')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE = 32
TARGET_SIZE = (224,224)
root = '/content/tiny-imagenet2/Tiny-ImageNet2'
train_root = '/content/tiny-imagenet2/Tiny-ImageNet2/train'
val_root = '/content/tiny-imagenet2/Tiny-ImageNet2/val'


datagen = ImageDataGenerator(rescale= 1./255)

#For img data folder
generator_train = datagen.flow_from_directory(
        train_root,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        interpolation="nearest")
generator_valid = datagen.flow_from_directory(
        val_root,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        interpolation="nearest")

steps_per_epoch = int(generator_train.n / BATCH_SIZE)
steps_valid = (generator_valid.n) // BATCH_SIZE
class_names = os.listdir(train_root)
class_names.sort()

## Training

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler,EarlyStopping,ReduceLROnPlateau

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 100:
        lr *= 0.5e-3
    elif epoch > 30:
        lr *= 1e-3
    elif epoch > 20:
        lr *= 1e-2
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=3,
                               min_lr=0.5e-6)
model_save_path = '/content/drive/My Drive/Slice-ImageNet'
model_name = model.name + '.hdf5'
model_path = os.path.join(model_save_path,model_name)
earlyStopping = EarlyStopping(monitor = 'val_loss',min_delta = 0,patience = 10,restore_best_weights = True)
mcp_save = ModelCheckpoint(model_path,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
callbacks = [mcp_save,lr_scheduler,lr_reducer]
# callbacks = [mcp_save]

In [ ]:
epochs1 = 30
print(model.name)
history0 = model.fit_generator(generator= generator_train,
                                   steps_per_epoch = steps_per_epoch,
                                   validation_data=generator_valid,
                                   epochs=epochs1, verbose=1, workers=1, validation_steps=steps_valid,
                                   callbacks=callbacks, use_multiprocessing=False)
with open(os.path.join('/content/drive/My Drive/Slice-ImageNet', model.name +'.pkl'), 'wb') as fp:
  pickle.dump(history0.history,fp,protocol=pickle.HIGHEST_PROTOCOL)
model_json = model.to_json()
with open(os.path.join('/content/drive/My Drive/Slice-ImageNet',model.name+'.json'), 'w') as json_file:
  json_file.write(model_json)

## Testing

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
 
def plot_training_history(history):
    plt.subplot(2, 1, 1)
    # Plot training & validation accuracy values
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('Model accuracy '+model.name)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()


    # Plot training & validation loss values
    plt.subplot(2, 1, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
plot_training_history(history0)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
def predict(loaded_model,img_dirs):
  count = 0
  for img_dir in img_dirs:
    img = image.load_img(img_dir, target_size= (224, 224))
    img = image.img_to_array(img)
    img /= 255
    img = np.expand_dims(img,axis=0)
    y_pred = loaded_model.predict(img)
    cls_pred = np.argmax(y_pred, axis=1)
    cls_pred = int(cls_pred[0])
    # print(cls_pred)
    class_predict = class_names[cls_pred]
    if class_predict in img_dir:
      count += 1
  return ((count/200) * 100)
def load_file(file_path):
  f = open(file_path,'rb')
  return f
def compareHis(arrHis,h1,h2):
    plt.subplot(2, 1, 1)
    # Plot validation accuracy values
    for history in arrHis:
      plt.plot(history['val_categorical_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend([h1, h2], loc='upper left')
    plt.show()

    # Plot validation loss values
    plt.subplot(2, 1, 1)
    for history in arrHis:
      plt.plot(history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend([h1, h2], loc='upper left')
    plt.show()
def get_new_model(model_path,json_path):
  with open(json_path,'r') as json_file:
    loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  # load weights into new model
  loaded_model.load_weights(model_path)
  return loaded_model

In [ ]:
result = model.evaluate_generator(generator=generator_valid, steps=steps_valid,verbose = 1)
print(model.name+" Valid-set classification accuracy: {0:.2%}".format(result[1]))

In [ ]:
img_dirs = glob.glob('/content/tiny-imagenet2/Tiny-ImageNet2/test/*.jpg')
img_dirs.sort()
print(model.name)
print(predict(model,img_dirs))